In [18]:
import pandas as pd
df = pd.read_csv(
    'UKFlora.csv', 
    sep=',', 
    low_memory=False, 
    usecols=['AccSpeciesName', 'OrigUnitStr', 'DataName', 'OrigValueStr'] # dont bother with columns we wont use

)

In [22]:
# attach units to data names in case we need them later
df['DataName'] = df['DataName'] + '(' + df['OrigUnitStr'] + ')'

In [64]:
type_map = {}

# make a dict mapping data name values to their types
# try float then string. don't bother with int as it won't work for columns with NaN values
# without messing about with deep pandas features: 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html#integer-na

for name, group in df.groupby('DataName'):
    for my_type in (float, str):
        try:
            coerced = group['OrigValueStr'].astype(my_type)
            type_map[name] = my_type
            break
        except:
            pass
type_map

{'Actual EvapoTranspiration(mm)': float,
 'Altitude (maximum recorded)(m)': str,
 'Altitude (minimum recorded)(m)': str,
 'Altitude (typical minimum)(m)': float,
 'Altitude(m)': float,
 'Average annual relative humidity(%)': float,
 'Average number of ground frost days per year (sum) (FRS)(d/y)': float,
 'Chromosome cDNA content(pg)': float,
 'Chromosome cDNA content(pg/2C-Nucleus)': float,
 'Cloud cover(%)': float,
 'Dispersal unit (dispersule / diaspore) length(mm)': str,
 'Ecosystem rooting depth(m)': float,
 'End of flowering(month)': float,
 'Fern  and moss spore width (diameter)(microm)': float,
 'Fine root diameter(microm)': str,
 'First historical record: 1. date(year)': float,
 'Flower pollen ovule ratio(1/Ovulo)': float,
 'Flowering periode: peak month(month)': float,
 'Fraction of absorbed photosynthetic active radiation (FAPAR) of the site(no unit)': float,
 'GPP of the site(gC/m2)': float,
 'Inbreeding(%)': str,
 'Intensity of mycorrhizal infection(%)': str,
 'Leaf Relativ

In [65]:
# takes a Series of values and aggregates them to a single value based on types in the dict
# for floats, take the median
# for strings, take the most common value
# replace this with your real logic :-)

def aggregate(values):
    species, data_name = values.name
    target_type = type_map[data_name]
    #print(data_name, target_type)

    coerced_data = values.astype(target_type)
    
    if target_type  == float:
        return coerced_data.median()
    else:
        return coerced_data.value_counts().index[0]
    
# make a summary table with the aggregated values
summary = df.groupby(['AccSpeciesName', 'DataName'])['OrigValueStr'].apply(aggregate).unstack()

In [66]:
# convert the columns in the summary table to the correct types
# using the dict from before

for c in summary.columns:
    summary[c] = summary[c].astype(type_map[c])

In [77]:
summary.dtypes

DataName
Actual EvapoTranspiration(mm)                                                                                                float64
Altitude (maximum recorded)(m)                                                                                                object
Altitude (minimum recorded)(m)                                                                                                object
Altitude (typical minimum)(m)                                                                                                float64
Altitude(m)                                                                                                                  float64
Average annual relative humidity(%)                                                                                          float64
Average number of ground frost days per year (sum) (FRS)(d/y)                                                                float64
Chromosome cDNA content(pg)                                 

In [76]:
# to figure out why certain columns aren't being sucessfully converted to numerical types
# find all the unique individual values in each data name that can't be converted to float

for name, group in df.groupby('DataName'):
    non_floats = set()
    for value in group['OrigValueStr']:
        try:
            float(value)
        except:
            non_floats.add(value)
    if len(non_floats) > 0:
        print(name)
        print(non_floats)
        print()

Altitude (maximum recorded)(m)
{'1,200'}

Altitude (minimum recorded)(m)
{'-'}

Dispersal unit (dispersule / diaspore) length(mm)
{'-'}

Fine root diameter(microm)
{'150-200', '<100', '100-150', '200-300', '300-500'}

Inbreeding(%)
{'90-95'}

Intensity of mycorrhizal infection(%)
{'rarely mycorrhizal', 'occasionally mycorrhizal', 'on soil only', 'not known in the wild', 'never mycorrhizal', 'rare', 'normally mycorrhizal'}

Leaf area: in case of compound leaves undefined if leaf or leaflet; undefined if petiole and rhachis in- or excluded(cm2)
{'>1000', '10-100', '1-10', '0.1-1', '100-1000', '<0.1'}

Leaf lamina thickness(mm)
{'>2.0', '0.5-1.0', '0.2-0.5', '<0.2', '1.0-2.0'}

Leaf lifespan (longevity, retention time, LL)(month)
{'12-24', '3-6', '6-12', '7-12', '2-6', '0-2', '>24'}

Leaf nitrogen content per dry mass (Nmass)(mg/g)
{'<5', '20-30', '5-10', '10-20', '>30'}

Leaf phosphorus content per dry mass (Pmass)(mg/g)
{'1-2', '<1', '2-3', '>3'}

Onset of flowering (first flowering dat

In [44]:
for labels, group in df.head(10000).groupby(['AccSpeciesName','DataName']):
    species, data = labels
    target_type = type_map[data]
    coerced_data = group['OrigValueStr'].astype(target_type)
    

ACAENA NOVAE-ZELANDIAE Altitude (minimum recorded)(m) ['0', '0']
ACAENA NOVAE-ZELANDIAE End of flowering(month) [7]
ACAENA NOVAE-ZELANDIAE First historical record: 1. date(year) [1901]
ACAENA NOVAE-ZELANDIAE Onset of flowering (first flowering date, beginning of flowering period)(month) ['6', '6']
ACAENA NOVAE-ZELANDIAE Plant height (unspecified if vegetative or reproductive)(cm) ['15', '2', '2']
ACAENA NOVAE-ZELANDIAE Root rooting depth (cm) ['0-10']
ACAENA NOVAE-ZELANDIAE Seed dry mass(mg) ['0.92']
ACAENA NOVAE-ZELANDIAE Seed length (largest dimension length)(mm) ['2.0']
ACAENA NOVAE-ZELANDIAE Seed number per plant(1/plant) ['1000-10000']
ACAENA NOVAE-ZELANDIAE Seeds per flower(1/flower) ['1']
ALISMA PLANTAGO-AQUATICA Altitude (maximum recorded)(m) ['381', '403']
ALISMA PLANTAGO-AQUATICA Altitude (minimum recorded)(m) ['0', '0', '0']
ALISMA PLANTAGO-AQUATICA Chromosome cDNA content(pg) [20.6]
ALISMA PLANTAGO-AQUATICA Dispersal unit (dispersule / diaspore) length(mm) ['2.4', '2.4']
AL

Agrostis vinealis Seedbank density(m-2 ?) ['69', '65', '67']
Agrostis vinealis Stem longevity(yrs) ['<1']
Ailanthus altissima Dispersal unit (dispersule / diaspore) length(mm) ['35']
Ailanthus altissima Leaf area: in case of compound leaves undefined if leaf or leaflet; undefined if petiole and rhachis in- or excluded(cm2) ['100-1000']
Ailanthus altissima Plant height (unspecified if vegetative or reproductive)(cm) ['2600']
Ailanthus altissima Pollen: 2. diameter (um)(microm) [25.0]
Ailanthus altissima Seed dry mass(mg) ['33.3']
Aira caryophyllea Altitude (maximum recorded)(m) ['563']
Aira caryophyllea Altitude (minimum recorded)(m) ['0', '0', '0', '0', '0']
Aira caryophyllea Chromosome cDNA content(pg) [12.1]
Aira caryophyllea First historical record: 1. date(year) [1605]
Aira caryophyllea Leaf area: in case of compound leaves undefined if leaf or leaflet; undefined if petiole and rhachis in- or excluded(cm2) ['<0.1']
Aira caryophyllea Maximum temperature of germination(C) [15.0]
Aira